# 用户标签学习目标
- 掌握用户标签的作用
- 使用SQL实现用户标签案例


---
  ```sql

#利用日期函数打上实时事实类标签
drop table if exists mall_customer_realtime_tag;
create table mall_customer_realtime_tag as
select a.userid,
       case when a.last_visit >= DATE_SUB(curdate(),INTERVAL 30 DAY) then "一个月活跃" else "近一个月不活跃" end as "近一个月活跃标签",
       case when a.last_visit >= DATE_SUB(curdate(),INTERVAL 90 DAY) then "近三个月活跃" else "近三个月不活跃" end as "近三个月活跃标签",
       case when a.last_visit >= DATE_SUB(curdate(),INTERVAL 180 DAY) then "近六个月活跃" else "近六个月不活跃" end as "近六个月活跃标签",
       case when a.last_visit >= DATE_SUB(curdate(),INTERVAL 360 DAY) then "近十二个月活跃" else "近十二个月不活跃" end as "近十二个月活跃标签"
from (
    select userid,max(date) as last_visit
    from mall_customers_tld_header
    group by userid
) a
group by userid;

# 年龄标签
select
    customerid,
    case when age <=18 then '青少年'
        when age between 19 and 24 then '青年'
        when age between 25 and 35 then '中青年'
        when age between 36 and 45 then '中年'
        when age between 46 and 55 then '中老年'
        else '老年'
    end as age_group_tag
from Mall_Customers;


# 基于收入
select customerid,
       case
           when Annual_Income>=50 then '高收入人群'
           else '普通收入人群'
       end as income_tag
from Mall_Customers;


# 消费能力标签
select
    customerid,
    case
        when Spending_Score>= 50 then '高消费人群'
        when Spending_Score between 11 and 49 then '普通消费人群'
        when Spending_Score=0 then '无消费人群'
        else '低消费人群'
    end as spending_tag
from Mall_Customers;

# 访问次数标签
select
    customerid,
    case
        when visits_p12>= 12 then '高频访客'
        when visits_p12 between 6 and 11 then '中频访客'
        else '低频访客'
    end as freq_tag
from Mall_Customers;

# 偏好标签
select
    customerid,
    case
        when morning_visits_p12/visits_p12> 0.5 then '早间访客'
        when afternoon_visit_p12/visits_p12>0.5 then '午间访客'
        when evening_visits_p12/visits_p12>0.5 then '晚间访客'
        else null
    end as daypart_preference_tag
from Mall_Customers;

select
    customerid,
    case
        when weekday_visits_p12/visits_p12> 0.5 then '周中访客'
        when weekend_visits_p12/visits_p12>0.5 then '周末访客'
        else null
    end as weekday_preference_tag
from Mall_Customers;

select
    customerid,
    case
        when with_kids_visits_p12/visits_p12> 0.5 then '热衷带娃'
        when with_couple_visits_p12/visits_p12>0.5 then '热衷恋爱'
        else null
    end as accompany_preference_tag
from Mall_Customers;

select
    customerid,
    case
        when clothing_spending_score/Spending_Score> 0.5 then '时尚达人'
        when foodbeverage_spending_score/Spending_Score>0.5 then '饮食男女'
        when entertainment_spending_score/Spending_Score>0.5 then '娱乐至上'
        when kids_spending_score/Spending_Score>0.5 then '鸡娃狂魔'
        else null
    end as target_preference_tag
from Mall_Customers;


# 新客老客
select
    customerid,
    case
        when first_visit_till_today <=30 then '新顾客'
        when first_visit_till_today>=180  then '老顾客'
        else '普通顾客'
    end as history_tag
from Mall_Customers;

# 所有标签汇总
drop table if exists mall_customer_aggregation_tags;
create table mall_customer_aggregation_tags as
select customerid,
    case
        when age <=18 then '青少年'
        when age between 19 and 24 then '青年'
        when age between 25 and 35 then '中青年'
        when age between 36 and 45 then '中年'
        when age between 46 and 55 then '中老年'
        else '老年'
    end                                                         as age_group_tag,
    case
        when Annual_Income>=50 then '高收入人群'
        else '普通收入人群'
    end                                                         as income_tag,
    case
        when Spending_Score>= 50 then '高消费人群'
        when Spending_Score between 11 and 49 then '普通消费人群'
        when Spending_Score=0 then '无消费人群'
        else '低消费人群'
    end                                                         as spending_tag,
    case
        when morning_visits_p12/visits_p12> 0.5 then '早间访客'
        when afternoon_visit_p12/visits_p12>0.5 then '午间访客'
        when evening_visits_p12/visits_p12>0.5 then '晚间访客'
        else null
    end                                                         as daypart_preference_tag,
    case
        when weekday_visits_p12/visits_p12> 0.5 then '周中访客'
        when weekend_visits_p12/visits_p12>0.5 then '周末访客'
        else null
    end                                                         as weekday_preference_tag,
    case
        when with_kids_visits_p12/visits_p12> 0.5 then '热衷带娃'
        when with_couple_visits_p12/visits_p12>0.5 then '热衷恋爱'
        else null
    end                                                         as accompany_preference_tag,
    case
        when clothing_spending_score/Spending_Score> 0.5 then '时尚达人'
        when foodbeverage_spending_score/Spending_Score>0.5 then '饮食男女'
        when entertainment_spending_score/Spending_Score>0.5 then '娱乐至上'
        when kids_spending_score/Spending_Score>0.5 then '鸡娃狂魔'
        else null
    end                                                         as target_preference_tag,
    case
        when first_visit_till_today <=30 then '新顾客'
        when first_visit_till_today>=180  then '老顾客'
        else '普通顾客'
    end                                                         as history_tag
from Mall_Customers;


SELECT *
FROM mall_customer_aggregation_tags;

# 将实时标签，预测类标签，统计类标签汇总
drop table if exists mall_customer_all_tags;
create table mall_customer_all_tags as
select
    a.*,
    b.age_group_tag,
    b.income_tag,
    b.spending_tag,
    b.daypart_preference_tag,
    b.weekday_preference_tag,
    b.accompany_preference_tag,
    b.target_preference_tag,
    b.history_tag
from mall_customer_realtime_tag a
    left join mall_customer_aggregation_tags b on a.userid=b.customerid;

# 找出最有消费潜力的用户
SELECT count(distinct userid) as customer_base
FROM mall_customer_all_tags
where income_tag='高收入人群' and spending_tag='低消费人群';

# 找出高收入人群中高消费人群的消费偏好
select
    count(distinct userid) as customer_base,
    target_preference_tag
from mall_customer_all_tags
where income_tag='高收入人群' and spending_tag='高消费人群'
group by target_preference_tag;

# 提取近一个月有过交易的普通消费人群
select distinct userid
from mall_customer_all_tags
where `近一个月活跃标签`='一个月活跃' and spending_tag='普通消费人群';

  ```


# 总结
- 事实类标签无需事先计算，仅需要开发一套实时查询的SQL通用语句
- 需要用到预测模型的标签必须事先在Python中完成训练并记录参数
- 将完成训练的模型参数融入SQL代码完成预测标签的编写
- 统计类标签需要制定比较合理的阈值才能将定性标签做得合情合理

